In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

: 

In [3]:
# Load the Caffe-based face detector
model_weights = "models/res10_300x300_ssd_iter_140000.caffemodel"
model_config = "models/deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(model_config, model_weights)

In [ ]:
# Load the pre-trained emotion classification model
emotion_model = load_model('models/emotion_model.h5')  # Replace with your model path
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
# Open the input video
video_path = 'face-demographics-walking-and-pause.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the frame dimensions
    (h, w) = frame.shape[:2]

    # Create a blob from the frame and pass it through the network
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], swapRB=False, crop=False)
    net.setInput(blob)
    detections = net.forward()

    # Loop over the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections
        if confidence > 0.7:  # Confidence threshold
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Extract the face region
            face_region = frame[startY:endY, startX:endX]

            # Preprocess the face for emotion classification
            face_resized = cv2.resize(face_region, (48, 48))  # Resize to model input size
            face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
            face_normalized = face_gray / 255.0
            face_input = np.expand_dims(np.expand_dims(face_normalized, axis=-1), axis=0)    

            # Predict emotion
            emotion_prediction = emotion_model.predict(face_input)
            emotion_label = emotion_labels[np.argmax(emotion_prediction)]

            # Draw bounding box and emotion label
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, emotion_label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)            

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows() 